# 1. 20 뉴스그룹 데이터 준비 및 특성 추출

20 뉴스그룹 데이터셋은 텍스트 마이닝에서 문서 분류의 성능을 측정하기 위해 가장 많이 사용되는 데이터셋 중 하나이다.

## 1.1 데이터셋 확인 및 분리

사이킷런 20 뉴스그룹 데이터는 다음과 같은 특징을 갖는다.
* categories 매개변수를 이용해 20개의 topic 중에서 원하는 토픽을 선택할 수 있다.
* remove로 필요없는 데이터를 삭제할 수 있다.
* 각 데이터셋 내에서 .data는 텍스트의 내용을, .target은 숫자로 표시된 라벨을 가져오는 데 사용된다.

In [1]:
from sklearn.datasets import fetch_20newsgroups

# 20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# 학습 데이터셋을 가져옴
newsgroups_train = fetch_20newsgroups(subset='train',
# 메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로 분류                                     
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

# 평가 데이터셋을 가져옴
newsgroups_test = fetch_20newsgroups(subset='test',
                                    remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

print('#Train set size:', len(newsgroups_train.data))
print('#Test set size:', len(newsgroups_test.data))
print('#Selected categories:', newsgroups_train.target_names)
print('#Train labels:', set(newsgroups_train.target))

#Train set size: 2034
#Test set size: 1353
#Selected categories: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
#Train labels: {0, 1, 2, 3}


데이터의 첫째 값을 확인해본다.

In [2]:
print('#Train set text samples:', newsgroups_train.data[0])
print('#Train set label samples:', newsgroups_train.target[0])
print('#Test set text samples:', newsgroups_test.data[0])
print('#Test set label samples:', newsgroups_test.target[0])

#Train set text samples: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
#Train set label samples: 1
#Test set text samples: TRry the SKywatch project in  Arizona.
#Test set label samples: 2


## 1.2 카운트 기반 특성 추출

In [3]:
X_train = newsgroups_train.data # 학습 데이터셋 문서
y_train = newsgroups_train.target # 학습 데이터셋 라벨

X_test = newsgroups_test.data # 평가 데이터셋 문서
y_test = newsgroups_test.target # 평가 데이터셋 라벨

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000, min_df=5, max_df=0.5)

X_train_cv = cv.fit_transform(X_train) # train set을 변환
print('Train set dim:', X_train_cv.shape)
X_test_cv = cv.transform(X_test) # test set을 변환
print('test set dimension:', X_test_cv.shape)

Train set dim: (2034, 2000)
test set dimension: (1353, 2000)


대략 어떤 단어들이 얼마나 있는지 보기 위해 단어의 빈도를 알아본다.

In [5]:
for word, count in zip(cv.get_feature_names_out()[:100], X_train_cv[0].toarray()[0,:100]):
    print(word, ':', count, end=', ')

00 : 0, 000 : 0, 01 : 0, 04 : 0, 05 : 0, 10 : 0, 100 : 0, 1000 : 0, 11 : 0, 12 : 0, 128 : 0, 129 : 0, 13 : 0, 130 : 0, 14 : 0, 15 : 0, 16 : 0, 17 : 0, 18 : 0, 19 : 0, 1987 : 0, 1988 : 0, 1989 : 0, 1990 : 0, 1991 : 0, 1992 : 0, 1993 : 0, 20 : 0, 200 : 0, 202 : 0, 21 : 0, 22 : 0, 23 : 0, 24 : 0, 25 : 0, 256 : 0, 26 : 0, 27 : 0, 28 : 0, 2d : 0, 30 : 0, 300 : 0, 31 : 0, 32 : 0, 33 : 0, 34 : 0, 35 : 0, 39 : 0, 3d : 0, 40 : 0, 400 : 0, 42 : 0, 45 : 0, 50 : 0, 500 : 0, 60 : 0, 600 : 0, 65 : 0, 70 : 0, 75 : 0, 80 : 0, 800 : 0, 90 : 0, 900 : 0, 91 : 0, 92 : 0, 93 : 0, 95 : 0, _the : 0, ability : 0, able : 1, abortion : 0, about : 1, above : 0, absolute : 0, absolutely : 0, ac : 0, accept : 0, acceptable : 0, accepted : 0, access : 0, according : 0, account : 0, accurate : 0, across : 0, act : 0, action : 0, actions : 0, active : 0, activities : 0, activity : 0, acts : 0, actual : 0, actually : 0, ad : 0, add : 0, added : 0, addition : 0, additional : 0, address : 0, 

# 2. 나이브 베이즈 분류기를 이용한 문서 분류 

In [6]:
from sklearn.naive_bayes import MultinomialNB

# 분류기 선언
NB_clf = MultinomialNB()

# train set을 이용해 분류기 학습
NB_clf.fit(X_train_cv, y_train)

# train_set에 대한 예측 정확도를 확인
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train)))

# test set에 대한 예측 정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))

Train set score: 0.824
Test set score: 0.732


# 3. 카운트 기반의 문제점과 N-gram을 이용한 보완

N-gram은 n개의 연속적인 단어들의 나열을 의미한다.

## 3.1 N-gram을 이용한 문서 분류

우선 unigram, bi-gram, tri-gram의 비교를 위해 아래와 같이 Unigram으로 TfidVectorizer 객체를 새로 생성하고 변환된 TF-IDF 벡터의 크기를 확인한다.

In [10]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

cachedStopWords = stopwords.words('english')
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}",
                       decode_error='ignore',
                       lowercase=True,
                       stop_words=stopwords.words('english'),
                       max_df=0.5,
                       min_df=2)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 11483)


주어진 인수 기준으로 토큰화했을 때는 남는 단어의 수가 11483개인 것을 확인했다.

N-gram을 사용하면 변수가 늘어나고 이로 인해 과적합의 우려가 있으므로 릿지 회귀분석을 사용한다.

In [11]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier()
ridge_clf.fit(X_train_tfidf, y_train)

# train_set에 대한 예측 정확도를 확인
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))

# test set에 대한 예측 정확도를 확인
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.976
Test set score: 0.766


이제 bi-gram을 적용해보자. TfidVectorizer의 ngram_range 매개변수를 사용하면 쉽게 N-gram을 적용할 수 있다.  
    ngram_range 매개변수는 시작 N값과 끝 N값으로 이루어진 튜플을 인수로 받는다. unigram 없이 학습했을 때 성능이 좋지 않을 것을 짐작할 수 있으므로 원래 변수에 bi-gram을 추가하는 것으로 한다.

In [13]:
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}",
                       decode_error='ignore',
                       lowercase=True,
                       stop_words=stopwords.words('english'),
                       ngram_range=(1, 2), # 바이그램 설정 
                       max_df=0.5,
                       min_df=2)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 26550)


변환된 결과를 보면 특성의 수가 26550으로 늘어난 것을 볼 수 있다.  
bi-gram이 어떤 식으로 생성되는지 확인하기 위해 아래와 같이 특성이름을 출력하고, 릿지 회귀분석으로 학습한 후 결과를 본다.

In [14]:
bigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 1]
print('bi-gram samples:', bigram_features[:10])

ridge_clf.fit(X_train_tfidf, y_train)

# train_set에 대한 예측 정확도를 확인
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))

# test set에 대한 예측 정확도를 확인
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

bi-gram samples: ["'cause can't", "'em better", "'expected errors'", "'karla' next", "'nodis' password", "'official doctrine", "'ok see", "'sci astro'", "'what's moonbase", 'aas american']
Train set score: 0.976
Test set score: 0.773


bi-gram의 효과가 약간은 있는 것으로 생각할 수 있다.